In [26]:
import pickle
import pandas as pds
import datetime

%load_ext autoreload

%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
file = pickle.load(open('py3dcore_h4c/custom_data/solo_2022_sep_mag_ll_swa_science.p', 'rb'))
data = pds.DataFrame(file)
#data['time'] = pds.to_datetime(data['time'])
#data.set_index('time',  inplace=True)
#data.index.name = None
#data.index = data.index.tz_localize(None)

In [28]:
data

,time,bx,by,bz,bt,r,lat,lon,x,y,z,vx,vy,vz,vt,tp,np
0,2022-09-01 00:00:00,48.886748,-0.938245,-12.110829,50.373273,0.748313,-2.947243,147.928511,0.0,0.0,0.0,609.740601,-52.652805,68.371964,615.817053,316229.160628,10.865103
1,2022-09-01 00:01:00,51.424539,-2.400445,-15.524266,53.770328,0.748306,-2.947290,147.928689,0.0,0.0,0.0,609.740601,-52.652805,68.371964,615.817053,316229.160628,10.865103
2,2022-09-01 00:02:00,51.528591,-2.066619,-15.495286,53.847660,0.748300,-2.947338,147.928867,0.0,0.0,0.0,609.086643,-32.513464,44.671487,611.587447,346301.909565,10.885287
3,2022-09-01 00:03:00,51.126701,0.204574,-14.495279,53.142211,0.748293,-2.947385,147.929045,0.0,0.0,0.0,589.783359,-22.259702,30.630256,590.997561,323740.627090,10.330824
4,2022-09-01 00:04:00,48.264353,0.488228,-13.773361,50.193542,0.748286,-2.947432,147.929223,0.0,0.0,0.0,592.967924,-5.329732,25.960779,593.559876,308713.845160,9.939938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15835,2022-09-11 23:55:00,3.169074,-17.743935,-2.050914,18.141018,0.635827,-4.803567,152.457813,0.0,0.0,0.0,591.793396,13.568009,-51.051800,594.146279,113410.173333,14.493595
15836,2022-09-11 23:56:00,3.187898,-17.810251,-1.978789,18.201190,0.635820,-4.803722,152.458261,0.0,0.0,0.0,591.793396,13.568009,-51.051800,594.146279,113410.173333,14.493595
15837,2022-09-11 23:57:00,3.067040,-18.052483,-1.758600,18.395422,0.635812,-4.803877,152.458710,0.0,0.0,0.0,591.793396,13.568009,-51.051800,594.146279,113410.173333,14.493595
15838,2022-09-11 23:58:00,2.925229,-18.103557,-1.541557,18.403047,0.635804,-4.804031,152.459158,0.0,0.0,0.0,591.793396,13.568009,-51.051800,594.146279,113410.173333,14.493595


In [40]:
import py3dcore_h4c

fitter = py3dcore_h4c.ABC_SMC()

model_kwargs = {
        "ensemble_size": int(2**17), #2**17
        "iparams": {
           "cme_longitude": {
               "maximum": -110,
               "minimum": -130
           },
           "cme_latitude": {
               "maximum": -10,
               "minimum": -50
           },
           "cme_inclination": {
               "maximum": 10,
               "minimum": 0
           }, 
            "cme_launch_velocity": {
                "maximum": 750,
                "minimum": 250
            },
            "cme_launch_radius": {
                "maximum": 30,
                "minimum": 5
            }
        }
    }

t_launch = datetime.datetime(2022, 9, 5, 18, 45, tzinfo=datetime.timezone.utc)

t_s_psp = datetime.datetime(2022, 9, 7, 0, 30, tzinfo=datetime.timezone.utc)
t_e_psp = datetime.datetime(2022, 9, 8, 5, 0,  tzinfo=datetime.timezone.utc)

t_psp = [
    datetime.datetime(2022, 9, 7, 2, tzinfo=datetime.timezone.utc),
    datetime.datetime(2022, 9, 7, 4, tzinfo=datetime.timezone.utc) ,
    datetime.datetime(2022, 9, 7, 6, tzinfo=datetime.timezone.utc),
    datetime.datetime(2022, 9, 7, 8, tzinfo=datetime.timezone.utc),
]
fitter.initialize(t_launch, py3dcore_h4c.ToroidalModel, model_kwargs)
fitter.add_observer("SOLO", t_psp, t_s_psp, t_e_psp)



In [41]:
fitter.observers

[['SOLO',
  [datetime.datetime(2022, 9, 7, 2, 0, tzinfo=datetime.timezone.utc),
   datetime.datetime(2022, 9, 7, 4, 0, tzinfo=datetime.timezone.utc),
   datetime.datetime(2022, 9, 7, 6, 0, tzinfo=datetime.timezone.utc),
   datetime.datetime(2022, 9, 7, 8, 0, tzinfo=datetime.timezone.utc)],
  datetime.datetime(2022, 9, 7, 0, 30, tzinfo=datetime.timezone.utc),
  datetime.datetime(2022, 9, 8, 5, 0, tzinfo=datetime.timezone.utc),
  None]]

In [43]:
dt_all = [t_s_psp] + t_psp + [t_e_psp]
from py3dcore_h4c.fitter.base import custom_observer

observer_obj = custom_observer('solo_2022_sep_mag_ll_swa_science.p')

data = observer_obj.get([t_s_psp, t_e_psp], "mag", reference_frame= 'HEEQ', sampling_freq=300, use_cache=True, as_endpoints=True)
tra = observer_obj.trajectory(dt_all, reference_frame='HEEQ', data_key = "pos")

[ 0.13914297  0.13910127  0.13905956 ... -0.05783126 -0.05811244
 -0.05839362]


In [44]:
tra

array([[ 0.2394255 , -0.30811386, -0.56808166],
       [ 0.25143756, -0.3072583 , -0.56255603],
       [ 0.26762182, -0.30532535, -0.55505135],
       [ 0.28395723, -0.30246706, -0.54739161],
       [ 0.3003972 , -0.29866356, -0.53957872],
       [ 0.46616585, -0.19870503, -0.44883025]])

In [42]:
from py3dcore_h4c.fitter.base import CustomData

data_obj = CustomData(fitter.observers, 'HEEQ','solo_2022_sep_mag_ll_swa_science.p')
data_obj.generate_noise("psd", 300)

[ 0.13914297  0.13910127  0.13905956 ... -0.05783126 -0.05811244
 -0.05839362]


In [35]:
import numpy as np

_ = np.linspace(t_s_psp.timestamp(), t_e_psp.timestamp(), int((t_e_psp.timestamp() - t_s_psp.timestamp()) // 300))  
dtp = [datetime.datetime.fromtimestamp(_, datetime.timezone.utc) for _ in _]
tt = [x.replace(tzinfo=None) for x in dtp]

In [25]:
,second=0, microsecond=0 [0]

[datetime.datetime(2022, 9, 1, 0, 0),
 datetime.datetime(2022, 9, 1, 0, 5),
 datetime.datetime(2022, 9, 1, 0, 10),
 datetime.datetime(2022, 9, 1, 0, 15),
 datetime.datetime(2022, 9, 1, 0, 20),
 datetime.datetime(2022, 9, 1, 0, 25),
 datetime.datetime(2022, 9, 1, 0, 30),
 datetime.datetime(2022, 9, 1, 0, 35),
 datetime.datetime(2022, 9, 1, 0, 40),
 datetime.datetime(2022, 9, 1, 0, 45),
 datetime.datetime(2022, 9, 1, 0, 50),
 datetime.datetime(2022, 9, 1, 0, 55),
 datetime.datetime(2022, 9, 1, 1, 0),
 datetime.datetime(2022, 9, 1, 1, 5),
 datetime.datetime(2022, 9, 1, 1, 10),
 datetime.datetime(2022, 9, 1, 1, 15),
 datetime.datetime(2022, 9, 1, 1, 20),
 datetime.datetime(2022, 9, 1, 1, 25),
 datetime.datetime(2022, 9, 1, 1, 30),
 datetime.datetime(2022, 9, 1, 1, 35),
 datetime.datetime(2022, 9, 1, 1, 40),
 datetime.datetime(2022, 9, 1, 1, 45),
 datetime.datetime(2022, 9, 1, 1, 50),
 datetime.datetime(2022, 9, 1, 1, 55),
 datetime.datetime(2022, 9, 1, 2, 0),
 datetime.datetime(2022, 9, 1,

In [42]:
ii = [np.where(file['time']==x)[0][0] for x in tt if np.where(file['time']==x)[0].size > 0]

In [43]:
ii

[0, 576, 1152, 1728, 2304, 2880]

In [27]:
data_obj.generate_noise("psd", 300)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/hruedisser/miniconda3/envs/newtest/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_25285/3518075560.py", line 1, in <module>
    data_obj.generate_noise("psd", 300)
  File "/home/hruedisser/py3DCORE_h4c/py3dcore_h4c/fitter/base.py", line 588, in generate_noise
    data = observer_obj.get([dt_s, dt_e], "mag", reference_frame=self.reference_frame, sampling_freq=sampling_freq, use_cache=True, as_endpoints=True)
  File "/home/hruedisser/py3DCORE_h4c/py3dcore_h4c/fitter/base.py", line 411, in get
    ii = [np.where(self.data['time']==x)[0][0] for x in tt]
  File "/home/hruedisser/py3DCORE_h4c/py3dcore_h4c/fitter/base.py", line 411, in <listcomp>
    ii = [np.where(self.data['time']==x)[0][0] for x in tt]
IndexError: index 0 is out of bounds for axis 0 with size 0

During handling of the above exception, another exception occurre

In [12]:
data_obj.generate_data([0])

In [9]:
from py3dcore_h4c.fitter.base import FittingData

data_obj = FittingData(fitter.observers, 'HEEQ') 

In [74]:
fitter.observers

[['PSP',
  [datetime.datetime(2022, 6, 2, 13, 0, tzinfo=datetime.timezone.utc),
   datetime.datetime(2022, 6, 2, 14, 0, tzinfo=datetime.timezone.utc),
   datetime.datetime(2022, 6, 2, 15, 0, tzinfo=datetime.timezone.utc),
   datetime.datetime(2022, 6, 2, 16, 0, tzinfo=datetime.timezone.utc)],
  datetime.datetime(2022, 6, 2, 12, 0, tzinfo=datetime.timezone.utc),
  datetime.datetime(2022, 6, 3, 6, 0, tzinfo=datetime.timezone.utc),
  None]]

In [50]:
t_psp = [
        datetime.datetime(2022, 9, 11, 0, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 6, 2, 14, tzinfo=datetime.timezone.utc) ,
        datetime.datetime(2022, 6, 2, 15, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 6, 2, 16, tzinfo=datetime.timezone.utc),
    ] 

In [87]:
data.loc[t_psp]

KeyError: "None of [DatetimeIndex(['2022-06-02 13:00:00+00:00', '2022-06-02 14:00:00+00:00',\n               '2022-06-02 15:00:00+00:00', '2022-06-02 16:00:00+00:00'],\n              dtype='datetime64[ns, UTC]', freq=None)] are in the [index]"

In [52]:
datetime.datetime.timestamp(t_psp[0])

1662854400.0

In [37]:
data[t_psp]

KeyError: "None of [DatetimeIndex(['2022-09-11 00:00:00+00:00', '2022-06-02 14:00:00+00:00',\n               '2022-06-02 15:00:00+00:00', '2022-06-02 16:00:00+00:00'],\n              dtype='datetime64[ns, UTC]', freq=None)] are in the [columns]"

In [38]:
data.index[0]

Timestamp('2022-09-01 00:00:00')